In [1]:
import yaml
from pathlib import Path
from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.data import DataStore
from gammapy.cube import SafeMaskMaker

In [2]:
str_ = """
    observations:
      datastore: $GAMMAPY_DATA/hess-dl3-dr1
      obs_cone:
        frame: icrs
        lon: 83.63 deg
        lat: 22.01 deg
        radius: 5 deg

    datasets:
      type: 3d
      stack: true
      geom:
        wcs:
          skydir:
            frame: icrs
            lon: 83.63 deg
            lat: 22.01 deg
          binsize: 0.02 deg
          fov:
            width: 3 deg
            height: 3 deg
          binsize_irf: 0.2 deg
          margin_irf: 0.5 deg
        selection:
          offset_max: 2.5 deg
        axes:
          energy:
            min: 0.1 TeV
            max: 100 TeV
            nbins: 24
          energy_true:
            min: 0.1 TeV
            max: 100 TeV
            nbins: 72
      map_selection: ['counts', 'exposure', 'background', 'psf', 'edisp']
"""
config = AnalysisConfig.from_yaml(str_)
analysis = Analysis(config)
analysis.get_observations()
analysis.get_datasets()


maker_safe_mask = SafeMaskMaker(methods=["edisp-bias", "bkg-peak"])
stacked = maker_safe_mask.run(analysis.datasets[0])

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}
Fetching observations.
Number of selected observations: 4
Creating geometry.
Creating datasets.
Processing observation 23592
Processing observation 23523
Processing observation 23526
Processing observation 23559
/home/luca/anaconda3/envs/gammapy-dev/lib/python3.7/site-packages/astropy/units/quantity.py:466: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [3]:
print(stacked.background_model.map.slice_by_idx({"energy":-2}).data.sum())
print(stacked.background_model.map.slice_by_idx({"energy":-1}).data.sum())

5.986328440957754
232881.39807494672
